In [12]:
import numpy as np
import cv2
import os
import time
import argparse

# Hardcoded argument values
image_path = r'C:\Users\91811\Desktop\python_ml\yolo algo\images\image1.jpg'
confidence_threshold = 0.5
threshold = 0.3

# Load the COCO class labels our YOLO model was trained on
labelsPath = 'yolo-coco/coco.names'
LABELS = open(labelsPath).read().strip().split("\n")

# Initialize a list of colors to represent each possible class label
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# Paths to the YOLO weights and model configuration
weightsPath = 'yolo-coco/yolov3.weights'
configPath = 'yolo-coco/yolov3.cfg'

# Load YOLO object detector
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# Load input image
image = cv2.imread(image_path)
(H, W) = image.shape[:2]

# Create blob from image
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)

# Set input blob for the network
net.setInput(blob)

# Get output layer indices
output_layer_indices = net.getUnconnectedOutLayers()

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in output_layer_indices]

# Forward pass
start = time.time()
layer_outputs = net.forward(output_layers)
end = time.time()

# Show timing information on YOLO
print("[INFO] YOLO took {:.6f} seconds".format(end - start))

# Initialize lists for detected bounding boxes, confidences, and class IDs
boxes = []
confidences = []
class_ids = []

# Process each layer output
for output in layer_outputs:
    # Process each detection
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter weak detections
        if confidence > confidence_threshold:
            # Scale the bounding box coordinates back relative to the
            # size of the image
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")

            # Use the center (x, y)-coordinates to derive the top
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # Update the bounding box coordinates, confidences, and
            # class IDs lists
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence, threshold)

# Ensure at least one detection exists
if len(idxs) > 0:
    # Loop over the indexes we are keeping
    for i in idxs.flatten():
        # Extract the bounding box coordinates
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        # Draw a bounding box rectangle and label on the image
        color = [int(c) for c in COLORS[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(LABELS[class_ids[i]], confidences[i])
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display the resulting image
cv2.imshow("Image with Bounding Boxes", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[INFO] YOLO took 1.324356 seconds
